In [1]:
from conn.connection import ConnectionManager

In [2]:
from utils import *

In [3]:
con = ConnectionManager()

20:51:29 SMART HOME/CONNECTION - ConnectionManager inicializado.
IP local: 192.168.18.179
IP do dispositivo (TCP): 192.168.18.189
Multicast: 239.1.1.1:7000
Porta TCP: 5006


In [4]:
def callback_fun(msg, add):
    print(msg,add)

In [5]:
con.start_tcp_server(callback_function=callback_fun)

20:51:29 SMART HOME/CONNECTION - Erro no servidor 192.168.18.189:5006: [WinError 10049] O endereço solicitado não é válido no contexto
20:51:29 SMART HOME/CONNECTION - Socket do servidor fechado.
